In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from Class_utils.parameters import file_paths,job_graph_par
from Class_utils.JobGraph import JobGraph
import ast
from Class_utils.FitnessFunctions import FitnessFunctions

%reload_ext autoreload
%autoreload 2

In [2]:
job_graph = JobGraph(file_paths, **job_graph_par)

Cache found loading...done


In [3]:
curricula = pd.read_csv("../outputs/0_curricula.csv").set_index("kId")

In [4]:
curricula

,qId,metadata,Education,Age,City,JobRange,Competences0,Competences1,Competences2,Competences3,...,Knowledge4,Knowledge5,Knowledge6,Language0,Language1,Language2,Language_level0,Language_level1,Language_level2,Experience
kId,,,,,,,,,,,,,,,,,,,,,
0,0,{'occ': 'b49f95f8-c1c8-428f-9f1f-5f5abae55e51'...,Master-D.,31,Belmonte Mezzagno,90,performing financial risk management in intern...,conform with deadlines,manage processes,concede business ethical code of conducts,...,-,-,-,Italian,French,English,A2,Null,Null,1
1,0,{'occ': 'b148e7a3-e5b0-4edc-b770-85ab81f36ac9'...,Master-D.,30,Casalmaggiore,80,perform financial risk management in internati...,managing a business with great care,build rapport with people from different cultu...,apply conflict management,...,embargo regulations,-,-,Italian,French,-,A2,Null,-,1
2,0,{'occ': 'fd00e4bb-dcee-41d7-81d3-2cdebad8fe70'...,Master-D.,40,Pozzuoli,80,produce sales reports,performing financial risk management in intern...,monitor international market performance,direct distribution operations,...,control export principles,embargo regulations,chemical products,French,-,-,Null,-,-,3
3,0,{'occ': 'bbba890e-ecae-43ba-9c5b-a65baa318f15'...,Master-D.,37,Vaiano,50,performing financial risk management in intern...,maintain financial records,-,-,...,-,-,-,Italian,French,-,A2,Null,-,4
4,0,{'occ': 'b2826761-6fc4-4344-9d36-d8455cb6b185'...,Master-D.,30,Lusciano,70,perform financial risk management in internati...,manage processes,demonstrate computer literacy,abide by business ethical code of conducts,...,export control principles,embargo legislation,animal health rules of distribution of product...,Italian,English,-,A2,A2,-,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19667,219,{'occ': 'f2413988-f76a-4636-a4b1-9952517aa21f'...,Degree,39,Marano di Napoli,70,perform risk analysis,perform maintenance on fire alarm systems,inspect fire equipment,ensure equipment availability,...,building codes,-,-,German,-,-,A2,-,-,2
19668,219,{'occ': '289c7385-2342-4a4e-b0b1-7b2e977aea79'...,Master-D.,18,Trani,30,understand the animal's situation,plan physical rehabilitation of animals,manage personal professional development,manage animal biosecurity,...,animal production science,anatomy of animals,-,Spanish,-,-,Null,-,-,2
19669,219,{'occ': '0264b794-3771-47aa-a675-78bf13b62ee7'...,Degree,32,Mondovi,60,translate keywords into full texts,revise legal document,perform clerical duties,manage personnel agenda,...,court procedures,civil law,-,Spanish,Italian,French,B2,Null,Null,1


In [5]:
curricula_md = pd.json_normalize(curricula["metadata"].apply(ast.literal_eval))
curricula_md.index.rename("kId", inplace=True)

In [6]:
# n_comp = curricula_md["uri_competences"].apply(lambda x:len(x))
# n_know = curricula_md["uri_knowledge"].apply(lambda x:len(x))
# 
# dt = pd.DataFrame({"n_comp":n_comp,"n_know":n_know})

In [8]:
comp_columns = ["Competences"+str(i) for i in range(0, 6 + 1)]
know_columns = ["Knowledge"+str(i) for i in range(0, 6 + 1)]

In [9]:
counter = np.zeros((2,5))
total = np.zeros((2,5))
qId_error_comp,qId_error_know = [],[]


for kId, cv in tqdm(curricula.iterrows(),total=len(curricula)):

    comp = FitnessFunctions.filter(cv[comp_columns].to_list())
    know = FitnessFunctions.filter(cv[know_columns].to_list())

    un_comp, amb_comp = job_graph.skill_standardize(comp)
    un_know, amb_know = job_graph.skill_standardize(know)

    contex = un_comp + un_know

    de_amb_comp = job_graph.solve_ambiguation(amb_comp, un_comp)
    de_amb_know = job_graph.solve_ambiguation(amb_know, un_know)

 
    real_uris = curricula_md.loc[kId, "uri_competences"]
    total[0, len(amb_comp)] += 1
    equal_ = 0 if set(un_comp) | set(de_amb_comp)  ==  set(real_uris) else 1
    if equal_ == 1:
        counter[0, len(amb_comp)] += 1
        qId_error_comp.append(kId)

    real_uris = curricula_md.loc[kId, "uri_knowledge"]
    total[1, len(amb_know)] += 1
    equal_ = 0 if set(un_know) | set(de_amb_know)  ==  set(real_uris) else 1
    if equal_ == 1:
        counter[1, len(amb_know)] += 1
        qId_error_know.append(kId)


100%|██████████| 19672/19672 [00:16<00:00, 1192.38it/s]


In [10]:
counter

array([[  0., 274.,  53.,   4.,   1.],
       [  0., 804., 312.,  30.,   2.]])

In [15]:
1 - counter.sum(axis=1)/19672

array([0.98312322, 0.94164294])

In [12]:
total

array([[1.6381e+04, 2.9780e+03, 2.9100e+02, 2.0000e+01, 2.0000e+00],
       [1.5790e+04, 3.3040e+03, 5.2600e+02, 5.0000e+01, 2.0000e+00]])

In [13]:
counter/total

array([[0.        , 0.09200806, 0.18213058, 0.2       , 0.5       ],
       [0.        , 0.2433414 , 0.59315589, 0.6       , 1.        ]])

In [14]:
# iiid = 171
# 
# essential_comp = FitnessFunctions.filter(offer.loc[iiid][es_comp_columns].to_list())
# print("Essential competence",essential_comp,"\n")
# optional_comp = FitnessFunctions.filter(offer.loc[iiid][op_comp_columns].to_list())
# print("Optional competence",optional_comp,"\n")
# essential_know = FitnessFunctions.filter(offer.loc[iiid][es_know_columns].to_list())
# print("Essential knowledge",essential_know,"\n")
# optional_know = FitnessFunctions.filter(offer.loc[iiid][op_know_columns].to_list())
# print("Optional knowledge", optional_know,"\n")
# 
# un_es_comp, amb_es_comp = job_graph.skill_standardize(essential_comp)
# un_op_comp, amb_op_comp = job_graph.skill_standardize(optional_comp)
# un_es_know, amb_es_know = job_graph.skill_standardize(essential_know)
# un_op_know, amb_op_know = job_graph.skill_standardize(optional_know)
# 
# contex = set(un_es_comp) | set(un_op_comp) | set(un_es_know) | set(un_op_know)
# 
# print("unique",un_es_comp,"\n")
# 
# print("ambigue",amb_es_comp,"\n")
# 
# deamb_es_comp = job_graph.solve_ambiguation(amb_es_comp, un_es_comp, True)
# # print("node similarity", job_graph.node_similarity(amb_op_comp[0], contex,True))
# 
# print("deambigued",deamb_es_comp,"\n")
# real_uri = offer_md.loc[iiid, "uri_competence_essential"]
# print("real",real_uri,"\n")
# print(set(un_es_comp) | set(deamb_es_comp) == set(real_uri))
# # essential_comp = FitnessFunctions.filter(job[es_comp_columns].to_list())
# # optional_comp = FitnessFunctions.filter(job[op_comp_columns].to_list())
# # essential_know = FitnessFunctions.filter(job[es_know_columns].to_list())
# # optional_know = FitnessFunctions.filter(job[op_know_columns].to_list())